In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


2025-04-13 08:36:09.609958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744533369.875220      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744533369.943721      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import pandas as pd
df=pd.read_csv('/kaggle/input/conversation-dataset/Conversation.csv')
df.head(20)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/conversation-dataset/Conversation.csv'

In [ ]:
df.drop(columns=['answer'], inplace=True)


In [ ]:
df.drop(df.columns[0], axis=1, inplace=True)

In [ ]:
df.to_csv('next_word.csv',index=False)

In [ ]:
df

In [ ]:
sentences = df['question'].to_string()  # Assuming 'question' is the column containing the sentences
sentences

In [ ]:
# Initialize Tokenizer
tokenizer = Tokenizer()

# Fit tokenizer on the sentences
tokenizer.fit_on_texts([sentences])

# Retrieve word index
word_index = tokenizer.word_index

# Print word index
print(len(word_index))

In [ ]:
 
#split my corpus into dox
embedded_list=[]
for sent in sentences.split('\n'):
  embeeded_dox=tokenizer.texts_to_sequences([sent])[0]
  if len(embeeded_dox ) !=0:
    # embedded_list.append(embeeded_dox)
    for i in range(1,len(embeeded_dox)):
      embedded_list.append(embeeded_dox[:i+1])
    
    


In [ ]:
max_len=max([len(i) for i in embedded_list])
max_len

In [ ]:
padded_embedded_list=pad_sequences(embedded_list,maxlen=max_len,padding='pre')


In [ ]:
X=[]
y=[]
for i in padded_embedded_list:
  X.append( i[:-1])
  y.append(i[-1])
X=np.asarray(X)
y=np.asarray(y)

In [ ]:
y

In [ ]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y)
y[1]


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=100 ))
model.add(LSTM(100))
model.add(Dense(6124, activation='softmax'))


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X, y, epochs=100, verbose=1)

In [ ]:
import time
text = "she"

for i in range(6):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)